### Setup your environnement variables

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

In [2]:
from dotenv import load_dotenv
assert load_dotenv("../.env.local", override=True) # Load environment variables from a .env file

# Get started

In [3]:
from uiform import UiForm
from uiform._utils.json_schema import filter_reasoning_fields_json
uiclient = UiForm()
models_list = uiclient.models.list()

/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Example 1 - Analyze informations from a document

#### Option A - You use our preprocess endpoint and openAI client

In [4]:
import json
from uiform import UiForm, Schema
from openai import OpenAI

with open("freight/schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = UiForm()
doc_msg = uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)



In [ ]:

# Now you can use your favorite model to analyze your document
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": schema_obj.id,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    }
)
print(completion.choices[0].message.content)

In [ ]:
# Validate the response against the original schema if you want to remove the reasoning fields
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate(
     filter_reasoning_fields_json(completion.choices[0].message.content)
)
extraction.model_dump()

In [5]:
print(json.dumps(schema_obj.inference_json_schema))

{"properties": {"reasoning___root": {"type": "string", "description": "Document your analysis of the overall booking confirmation. Include:\n\n    1. Document type identification and confidence\n    2. Key parties involved (client, carrier, etc.)\n    3. Overall structure of the shipments\n    4. Critical booking details and requirements\n    5. Any ambiguities or unclear elements\n\n    Example: \"Analysis shows:\n    - Clear booking confirmation document from ABC Corp\n    - Contains 2 distinct shipments under single booking reference\n    - Client identified as ABC Corp from letterhead\n    - Total value and payment terms clearly stated\n    - All required signatures present\"\n\n    Explain your reasoning for key decisions made in interpreting the document structure."}, "booking_id": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Unique identifier for the booking used for billing reference\nUser Provided a Default Value: null", "title": "Booking Id"}, "payment": 

In [6]:
print(json.dumps(schema_obj.inference_pydantic_model.model_json_schema()))

{"$defs": {"ClientData": {"additionalProperties": false, "properties": {"company_name": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Legal company name for billing\nUser Provided a Default Value: null", "title": "Company Name"}, "reasoning___VAT_number": {"description": "The VAT number is present for billing and tax purposes. Usually located at the top or the very bottom of the document, and located alongside the other billing informations (small text).\nIn France the VAT number format is FR {2 digits validation key} {first 9 digits SIREN number} so try use the SIREN number if available to help validate the VAT number.\nDocument your reasoning for the VAT number identification:\n1. Look for numbers prefixed with 'FR' or labeled as 'TVA', 'VAT', or 'n\u00b0 TVA'\n2. Verify the format matches FR followed by key of 2 digits and 9 digits of SIREN number\n3. Note where you found it in the document\n\nExample: \"Found VAT number FR12345678901 in footer next to SIREN numb

In [7]:

# Now you can use your favorite model to analyze your document
client = OpenAI()
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)
print(completion.choices[0].message.parsed)

reasoning___root='Analysis shows:\n- Clear booking confirmation document from ACME Corporation\n- Contains 1 distinct shipment under single booking reference\n- Client identified as ACME Corporation from letterhead\n- Total value and payment terms clearly stated\n- All required signatures present' booking_id='BC-67890' payment=TransportPriceData(total_price=1500.0, currency='EUR') client=ClientData(company_name='ACME Corporation', reasoning___VAT_number='Found VAT number GB123456789 in header next to company name', VAT_number='GB123456789', city='London', postal_code='WC2N 5DU', country='GB', code='SHIP-001', email='client@acme.com') reasoning___shipments='Shipment Analysis:\n1: 500 kg, 5 pallets, 10 units, 1.44 m³ (UN1202, Class 3 - Liquid flammable)\n2: Pickup from ACME Corporation on 5/2/2023\n3: Delivery to Beta Industries on 3/5/2023' shipments=[ShipmentData(shipment_id='SHIP-001', reasoning___sender='I have found within the documents the following information about the sender:\n-

In [ ]:
## If you want to log the extraction (keep it in our database)
# Limitation: we don't get the likelihoods
# For now one might send the messages in UiForm-compatible format.
uiclient.documents.extractions.log(
    document = "freight/booking_confirmation.jpg",
    messages = schema_obj.messages + doc_msg.messages,
    completion = completion,
    json_schema = json_schema,
    model = "gpt-4o-mini",
    temperature = 0,
)

In [ ]:
# You can use the messages from different providers (but you cannot mix them)
out_log_openai = uiclient.documents.extractions.log(
    document = "freight/booking_confirmation.jpg",
    openai_messages = schema_obj.openai_messages + doc_msg.openai_messages,
    completion = completion,
    json_schema = json_schema,
    model = "gpt-4o-mini",
    temperature = 0,
)
print(out_log_openai)

out_log_anthropic = uiclient.documents.extractions.log(
    document = "freight/booking_confirmation.jpg",
    anthropic_messages = schema_obj.anthropic_messages + doc_msg.anthropic_messages,
    anthropic_system_prompt = schema_obj.anthropic_system_prompt,
    completion = completion,
    json_schema = json_schema,
    model = "gpt-4o-mini",
    temperature = 0,
)
print(out_log_anthropic)





#### Option B - Using UiForm `extract` endpoint

In [ ]:
import json
from uiform.client import UiForm

uiclient = UiForm()
completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0,
)

filter_reasoning_fields_json(completion.choices[0].message.content)

# For streaming:
# from IPython.display import clear_output, display

# with uiclient.documents.extractions.stream(
#     json_schema = "freight/schema.json",
#     document = "freight/booking_confirmation.jpg",
#     model="gpt-4o-mini",
#     temperature=0,
# ) as stream:
#     for chunk in stream:
#         clear_output(wait=True)
#         display(chunk.model_dump())
        


## Appendix A - Use text as modality

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

## Appendix B - Add image settings

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

image_settings = {
    'dpi': 72,
    'browser_canvas': 'A4', 
    'image_to_text': 'ocr',
    'correct_image_orientation': True
}


completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    image_settings=image_settings
)

print(completion.choices[0].message.parsed)

## Appendix C - Use o3-mini model with reasoning effort

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

image_settings = {
    'dpi': 72,
    'browser_canvas': 'A4', 
    'image_to_text': 'ocr',
    'correct_image_orientation': True
}


completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="o3-mini",
    temperature=0,
    reasoning_effort="low",
    image_settings=image_settings
)

print(completion.choices[0].message.parsed)

## Appendix D - Consensus extraction

You can also benefit from consensus by running multiple extractions (with non-zero temperature) and gathering the best results with a more reliable likelihoods.

Be careful, the cost of consensus is as much higher as the number of consensus model you run. ($Total = N * Cost(model)$)

In [ ]:
from uiform.client import UiForm

uiclient = UiForm()

completion = uiclient.documents.extractions.parse(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0.5,
    # image_settings=image_settings,
    n_consensus=5
)

filter_reasoning_fields_json(completion.choices[0].message.content or "{}")

## Appendix E - Using OpenAI Responses API

In [5]:
from openai import OpenAI

# Now you can use your favorite model to analyze your document
client = OpenAI()

responses_output = client.responses.create(
    model="gpt-4o-mini",
    input=schema_obj.openai_responses_input + doc_msg.openai_responses_input,
    text={
        "format": {
            "type": "json_schema",
            "name": schema_obj.id,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    },
    temperature=0,
    store=False
)
print(responses_output.output_text)




{"reasoning___root":"Analysis shows:\n- Clear booking confirmation document from ACME Corporation.\n- Client identified as ACME Corporation in header with VAT number.\n- Document references a pickup and delivery schedule with transport requirements.\n- Payment terms clearly stated as 1500 EUR HT.\n- All required signatures and contact details present.","booking_id":"BC-67890","payment":{"total_price":1500,"currency":"EUR"},"client":{"company_name":"ACME Corporation","reasoning___VAT_number":"Found VAT number GB123456789 in header next to company name.","VAT_number":"GB123456789","city":"London","postal_code":"WC2N 5DU","country":"GB","code":null,"email":"client@acme.com"},"reasoning___shipments":"Document outlines one shipment with detailed descriptions and specific transport conditions.\n- The shipment contains dangerous goods classified as UN1202 Class 3 - Inflammable liquid.\n- Sender and recipient information includes comprehensive details regarding addresses and contact numbers.\n

In [6]:
# IF you want to keep it with us:
uiclient.documents.extractions.log(
    document = "freight/booking_confirmation.jpg",
    openai_responses_input = schema_obj.openai_responses_input + doc_msg.openai_responses_input,
    openai_responses_output = responses_output,
    json_schema = json_schema,
    model = "gpt-4o-mini",
    temperature = 0,
)

{'extraction_id': 'extr_hhVsxNsRZH1vQ7ATuODVj',
 'status': 'success',
 'error_message': None}

In [ ]:
{
    "detail": [
        {
            "type": "literal_error",
            "loc": ["body", "openai_responses_output", "text", "format", "ResponseFormatText", "type"],
            "msg": "Input should be 'text'",
            "input": "json_schema",
            "ctx": {"expected": "'text'"},
        },
        {
            "type": "missing",
            "loc": ["body", "openai_responses_output", "text", "format", "ResponseFormatTextJSONSchemaConfig", "schema"],
            "msg": "Field required",
            "input": {
                "schema_": {
                    "properties": {
                        "reasoning___root": {
                            "type": "string",
                            "description": "Document your analysis of the overall booking confirmation. Include:\n\n    1. Document type identification and confidence\n    2. Key parties involved (client, carrier, etc.)\n    3. Overall structure of the shipments\n    4. Critical booking details and requirements\n    5. Any ambiguities or unclear elements\n\n    Example: \"Analysis shows:\n    - Clear booking confirmation document from ABC Corp\n    - Contains 2 distinct shipments under single booking reference\n    - Client identified as ABC Corp from letterhead\n    - Total value and payment terms clearly stated\n    - All required signatures present\"\n\n    Explain your reasoning for key decisions made in interpreting the document structure.",
                        },
                        "booking_id": {
                            "anyOf": [{"type": "string"}, {"type": "null"}],
                            "description": "Unique identifier for the booking used for billing reference\nUser Provided a Default Value: null",
                            "title": "Booking Id",
                        },
                        "payment": {
                            "properties": {
                                "total_price": {
                                    "anyOf": [{"type": "number"}, {"type": "null"}],
                                    "description": "Net price of the booking order (excluding taxes)\nUser Provided a Default Value: null",
                                    "title": "Total Price",
                                },
                                "currency": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "Three-letter currency code (ISO 4217).\nUser Provided a Default Value: null",
                                    "title": "Currency",
                                },
                            },
                            "title": "TransportPriceData",
                            "type": "object",
                            "description": "Payment data",
                            "required": ["total_price", "currency"],
                            "additionalProperties": false,
                        },
                        "client": {
                            "properties": {
                                "company_name": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "Legal company name for billing\nUser Provided a Default Value: null",
                                    "title": "Company Name",
                                },
                                "reasoning___VAT_number": {
                                    "type": "string",
                                    "description": "The VAT number is present for billing and tax purposes. Usually located at the top or the very bottom of the document, and located alongside the other billing informations (small text).\nIn France the VAT number format is FR {2 digits validation key} {first 9 digits SIREN number} so try use the SIREN number if available to help validate the VAT number.\nDocument your reasoning for the VAT number identification:\n1. Look for numbers prefixed with 'FR' or labeled as 'TVA', 'VAT', or 'n° TVA'\n2. Verify the format matches FR followed by key of 2 digits and 9 digits of SIREN number\n3. Note where you found it in the document\n\nExample: \"Found VAT number FR12345678901 in footer next to SIREN number\"\n\nIf multiple possible VAT numbers exist, explain your choice. If none found, state \"No VAT number identified in document.",
                                },
                                "VAT_number": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "VAT number Value\nUser Provided a Default Value: null",
                                    "title": "Vat Number",
                                },
                                "city": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "City, district, suburb, town, or village of billing address\nUser Provided a Default Value: null",
                                    "title": "City",
                                },
                                "postal_code": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "ZIP or postal code of billing address. If french postal code, it should be a pure number, without letters.  It can be incomplete if the postal code is not fully visible, but should contain at least two digits.\nUser Provided a Default Value: null",
                                    "title": "Postal Code",
                                },
                                "country": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "Two-letter country code (ISO 3166-1 alpha-2).\nUser Provided a Default Value: null",
                                    "title": "Country",
                                },
                                "code": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "Unique code for the client used to connect the booking to the client in the TMS. It is rarely provided. If not provided, set to None\nUser Provided a Default Value: null",
                                    "title": "Code",
                                },
                                "email": {
                                    "anyOf": [{"type": "string"}, {"type": "null"}],
                                    "description": "Client email address\nUser Provided a Default Value: null",
                                    "title": "Email",
                                },
                            },
                            "title": "ClientData",
                            "type": "object",
                            "description": "Client data used for billing",
                            "required": ["company_name", "reasoning___VAT_number", "VAT_number", "city", "postal_code", "country", "code", "email"],
                            "additionalProperties": false,
                        },
                        "reasoning___shipments": {
                            "type": "string",
                            "description": "Document your analysis for each shipment. For each shipment identified in the document, include:\n\n1. A detailed description of the goods being transported, including any reference numbers, dangerous goods information, and packing details.\n2. The sender’s information, including the company name, address, contact details, and any relevant observations about the pickup process.\n3. The recipient’s information, including the company name, address, contact details, and any relevant observations about the delivery process.\n4. Any trucking constraints, including vehicle requirements, loading/unloading equipment, and specific transport conditions.\n\nExample:\nShipment Analysis:\n1: SSP BOC - 4 EUR pallets, 10ML, 7,623 kg  (UN 3082, liquid, 9, GE III)\n2: AMC(31) BOS 2249 - 8 pallets, 7ML, 3,240 kg\n3: VETISOL BOS 2195 - 10 pallets, 9ML, 3,780 kg\n4: VETISOL BOS 2282 - 6 pallets, 5ML, 2,020 kg\n5: SOERMEL BOS 2264 - 2 pallets, 8ML, 1,300 kg (UN 3077, solid, 9, GE III)\n6: DUMOULIN BOS 2281 - 15 pallets, 11ML, 10,400 kg\n7: SOLUSTIL BOS 1764 - 20 pallets, 12ML, 24,800 kg Refrigerated vehicle (-20°C)\n8: TECMA TAR 323 - 1 pallet, 2ML, 1,475 kg\n9: PAILLET BOS 2290 - 8 pallets, 10ML, 4,945 kg\n\nEnsure that each shipment is clearly detailed with all relevant information. Do not omit any critical details.\nMost frequently, there is going to be one or two shipments in the document, but sometimes you can have a massive number of shipments on several lines, more like a table containing a long list of shipments.\nIn this case, you should create a shipment for each line of the table, it can go to 10-20 shipments sometimes.\n",
                        },
                        "shipments": {
                            "description": "List of shipment data. The number of entries should correspond to the total loading/delivery point combinations identified in the document.",
                            "items": {
                                "properties": {
                                    "shipment_id": {
                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                        "description": "Unique identifier for the shipment, most often the CMR number or CMR reference, used for tracking and documenting transport details. Can also be client / sender reference number. If several numbers are provided and the situation is ambiguous, pick the most logical one, and put other numbers in the observation field.\nUser Provided a Default Value: null",
                                        "title": "Shipment Id",
                                    },
                                    "reasoning___sender": {
                                        "type": "string",
                                        "description": "Document your reasoning for the sender data of this shipment. You should, for example, list all the relevant information found about this shipment's sender, such as:\n\n- Sender company name\n- Sender address on the format: LINE1: [line1] / LINE2: [line2] / CITY: [city] / POSTAL_CODE: [postal_code] / COUNTRY: [country]\n- Sender phone number\n- Sender email\n- Sender pickup date and time\n- Sender observations\n\n**Important**: For the address info, try your best even if the data is incomplete (e.g., an invalid postal code with just two digits is better than nothing).\n\nThis reasoning sandbox should start like: 'I have found within the documents the following information about ...'\n ",
                                    },
                                    "sender": {
                                        "properties": {
                                            "company_name": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Name of the company.\nUser Provided a Default Value: null",
                                                "title": "Company Name",
                                            },
                                            "address": {
                                                "properties": {
                                                    "city": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "City, district, suburb, town, or village.\nUser Provided a Default Value: null",
                                                        "title": "City",
                                                    },
                                                    "postal_code": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "ZIP or postal code. If french postal code, it should be a pure number, without letters. It can be incomplete if the postal code is not fully visible, but should contain at least two digits.\nUser Provided a Default Value: null",
                                                        "title": "Postal Code",
                                                    },
                                                    "country": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Two-letter country code (ISO 3166-1 alpha-2).\nUser Provided a Default Value: null",
                                                        "title": "Country",
                                                    },
                                                    "line1": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Address line 1 (e.g., street, PO Box, or company name).\nUser Provided a Default Value: null",
                                                        "title": "Line1",
                                                    },
                                                    "line2": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Address line 2 (e.g., apartment, suite, unit, or building).\nUser Provided a Default Value: null",
                                                        "title": "Line2",
                                                    },
                                                },
                                                "title": "AddressDataSimple",
                                                "type": "object",
                                                "description": "Address of the sender.",
                                                "required": ["city", "postal_code", "country", "line1", "line2"],
                                                "additionalProperties": false,
                                            },
                                            "phone_number": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Phone number of the sender.\nUser Provided a Default Value: null",
                                                "title": "Phone Number",
                                            },
                                            "email_address": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Email address of the sender.\nUser Provided a Default Value: null",
                                                "title": "Email Address",
                                            },
                                            "pickup_datetime": {
                                                "properties": {
                                                    "date": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Date of the pickup. ISO 8601 Date Format: YYYY-MM-DD\nUser Provided a Default Value: null",
                                                        "title": "Date",
                                                    },
                                                    "start_time": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Start time of the pickup. ISO 8601 Time Format: hh:mm:ss\nUser Provided a Default Value: null",
                                                        "title": "Start Time",
                                                    },
                                                    "end_time": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "End time of the pickup. ISO 8601 Time Format: hh:mm:ss. Must be greater than or equal to the start_time.\nUser Provided a Default Value: null",
                                                        "title": "End Time",
                                                    },
                                                },
                                                "title": "PickupDatetimeData",
                                                "type": "object",
                                                "description": "pickup date and time.",
                                                "required": ["date", "start_time", "end_time"],
                                                "additionalProperties": false,
                                            },
                                            "observations": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Include all relevant information for pickup and sender details. This includes:\n\t•\tReference numbers (e.g., order or booking refs)\n\t•\tContact details (e.g., names, roles, extensions)\n\t•\tDock/door numbers, access codes, and navigation instructions\n\t•\tRequired documentation and special equipment needs\n\t•\tLoading bay specs, timing preferences, and site access instructions\n\t•\tParking info, specific entrance locations, and any additional sender notes\nWhen you write observations, write them in the same language as the email and the documents. If they are in French, write them in French, if they are in English, write them in English.\nUser Provided a Default Value: null",
                                                "title": "Observations",
                                            },
                                        },
                                        "title": "SenderData",
                                        "type": "object",
                                        "description": "Sender data",
                                        "required": ["company_name", "address", "phone_number", "email_address", "pickup_datetime", "observations"],
                                        "additionalProperties": false,
                                    },
                                    "reasoning___recipient": {
                                        "type": "string",
                                        "description": "Document your reasoning for the Recipient data of this shipment. You should, for example, list all the relevant information found about this shipment's Recipient, such as:\n\n- Recipient company name\n- Recipient address on the format: LINE1: [line1] / LINE2: [line2] / CITY: [city] / POSTAL_CODE: [postal_code] / COUNTRY: [country]\n- Recipient phone number\n- Recipient email\n- Recipient pickup date and time\n- Recipient observations\n\n**Important**: For the address info, try your best even if the data is incomplete (e.g., an invalid postal code with just two digits is better than nothing).\n\nThis reasoning sandbox should start like: 'I have found within the documents the following information about ...'\n ",
                                    },
                                    "recipient": {
                                        "properties": {
                                            "company_name": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Name of the company.\nUser Provided a Default Value: null",
                                                "title": "Company Name",
                                            },
                                            "address": {
                                                "properties": {
                                                    "city": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "City, district, suburb, town, or village.\nUser Provided a Default Value: null",
                                                        "title": "City",
                                                    },
                                                    "postal_code": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "ZIP or postal code. If french postal code, it should be a pure number, without letters. It can be incomplete if the postal code is not fully visible, but should contain at least two digits.\nUser Provided a Default Value: null",
                                                        "title": "Postal Code",
                                                    },
                                                    "country": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Two-letter country code (ISO 3166-1 alpha-2).\nUser Provided a Default Value: null",
                                                        "title": "Country",
                                                    },
                                                    "line1": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Address line 1 (e.g., street, PO Box, or company name).\nUser Provided a Default Value: null",
                                                        "title": "Line1",
                                                    },
                                                    "line2": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Address line 2 (e.g., apartment, suite, unit, or building).\nUser Provided a Default Value: null",
                                                        "title": "Line2",
                                                    },
                                                },
                                                "title": "AddressDataSimple",
                                                "type": "object",
                                                "description": "Address of the recipient.",
                                                "required": ["city", "postal_code", "country", "line1", "line2"],
                                                "additionalProperties": false,
                                            },
                                            "phone_number": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Phone number of the recipient.\nUser Provided a Default Value: null",
                                                "title": "Phone Number",
                                            },
                                            "email_address": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Email address of the recipient.\nUser Provided a Default Value: null",
                                                "title": "Email Address",
                                            },
                                            "delivery_datetime": {
                                                "properties": {
                                                    "date": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Date of the delivery. ISO 8601 Date Format: YYYY-MM-DD\nUser Provided a Default Value: null",
                                                        "title": "Date",
                                                    },
                                                    "start_time": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Start time of the delivery. ISO 8601 Time Format: hh:mm:ss\nUser Provided a Default Value: null",
                                                        "title": "Start Time",
                                                    },
                                                    "end_time": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "End time of the delivery. ISO 8601 Time Format: hh:mm:ss. Must be greater than or equal to the start_time.\nUser Provided a Default Value: null",
                                                        "title": "End Time",
                                                    },
                                                },
                                                "title": "DeliveryDatetimeData",
                                                "type": "object",
                                                "description": "delivery date and time. Must be after the pickup date and time.",
                                                "required": ["date", "start_time", "end_time"],
                                                "additionalProperties": false,
                                            },
                                            "observations": {
                                                "anyOf": [{"type": "string"}, {"type": "null"}],
                                                "description": "Include all relevant delivery information. This includes:\n\t•\tReference numbers (e.g., delivery refs, order numbers)\n\t•\tContact details (e.g., names, roles, extensions)\n\t•\tDock/door numbers, access codes, and specific entrance instructions\n\t•\tRequired documentation and any special equipment needs\n\t•\tUnloading bay specs, timing preferences, and site access instructions\n\t•\tParking info, unloading procedures, safety protocols, and any additional notes near recipient info\nWhen you write observations, write them in the same language as the email and the documents. If they are in French, write them in French, if they are in English, write them in English.\nUser Provided a Default Value: null",
                                                "title": "Observations",
                                            },
                                        },
                                        "title": "RecipientData",
                                        "type": "object",
                                        "description": "Recipient data",
                                        "required": ["company_name", "address", "phone_number", "email_address", "delivery_datetime", "observations"],
                                        "additionalProperties": false,
                                    },
                                    "reasoning___goods": {
                                        "type": "string",
                                        "description": "Document your analysis of the goods information from the source document. Include:\n\n1. Description of what's being transported\n2. Any reference numbers or codes found\n3. Special handling requirements\n4. Equipment needs\n5. Safety considerations\n6. Documentation requirements\n7. The presence of returnable Euro pallets (it has to be explicitly stated that the pallets are EU pallets in the source document in the sender section (not in the legal mentions))\n\n*Example of thinking output:*\n```\nSource indicates: \n- 12 pallets of automotive parts\n- Reference: ORDER123, BATCH456\n- Requires forklift for unloading\n- Non-stackable items\n- Fragile goods warning labels needed\"\n- Weight is in displayed in tons - its value is 5.4 tons, which is equivalent to 5400 kg.\n```\nList all relevant details found in the source document. Do not make assumptions about information that isn't explicitly stated.",
                                    },
                                    "goods": {
                                        "properties": {
                                            "packing": {
                                                "properties": {
                                                    "units": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "Number of units\nUser Provided a Default Value: null",
                                                        "title": "Units",
                                                    },
                                                    "packing_type": {
                                                        "anyOf": [{"enum": ["pallet", "bulk", "container", "other"], "type": "string"}, {"type": "null"}],
                                                        "description": "Type of packing - make sure you don't say it's a pallet (e.g. it says PAL) if you are not sure\nUser Provided a Default Value: null",
                                                        "title": "Packing Type",
                                                    },
                                                    "supplementary_parcels": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "Number of additional parcels\nUser Provided a Default Value: null",
                                                        "title": "Supplementary Parcels",
                                                    },
                                                    "reasoning___pallets_on_ground": {
                                                        "type": "string",
                                                        "description": "This field calculates the equivalent floor area occupied by packing units, expressed in terms of standard EUR pallets (1.2m × 0.8m). Follow these steps for each pallet, then sum the results to get the total floor pallet equivalent:\n\n\t1.\tFor each pallet: (length × width) / (1.2 × 0.8).\n\t2.\tSum the calculations to get the total floor pallet equivalent area.\n\nExample Calculations:\n\nExample 1: 6 EUR Pallets (Standard Size)\n\nIf all 6 pallets are standard EUR pallets:\n\n\t•\tEach pallet occupies: (1.2 × 0.8) / (1.2 × 0.8) = 1 pallet equivalent.\n\t•\tTotal for 6 pallets: 1 + 1 + 1 + 1 + 1 + 1 = 6.\n\nSo, floor pallet equivalent = 6.\n\nExample 2: Two Custom Pallets\n\n\t•\tPallet 1: 1.5m × 0.9m, so 1.5 × 0.9 / (1.2 × 0.8) ≈ 1.41.\n\t•\tPallet 2: 1.0m × 1.1m, so 1.0 × 1.1 / (1.2 × 0.8) ≈ 1.15.\n\nTotal floor pallet equivalent = 1.41 + 1.15 = 2.56.\n\nFor each pallet, write down the formula, insert values, and compute explicitly.\nTake your time to think rigourously. Do not hesitate to write down intermediate steps.",
                                                    },
                                                    "pallets_on_ground": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "pallets on ground value\nUser Provided a Default Value: null",
                                                        "title": "Pallets On Ground",
                                                    },
                                                    "number_eur_pallet": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "Number of pallets that follow the European standard, i.e. marked as something like EUR or EPAL. If not explicitly mentioned, set to 0.\nUser Provided a Default Value: null",
                                                        "title": "Number Eur Pallet",
                                                    },
                                                    "observation": {
                                                        "anyOf": [{"type": "string"}, {"type": "null"}],
                                                        "description": "Information about goods, packing, and handling requirements. This includes:\n\t•\tReference numbers (e.g., product codes, serial numbers)\n\t•\tHandling instructions, stacking limits, and loading specs (e.g., side/rear-loading)\n\t•\tVehicle, equipment, or packaging needs\n\t•\tSpecial care (e.g., temperature control, fragility, safety warnings)\n\t•\tLabels, placards, seal/container numbers, or other markings\nWhen you write observations, write them in the same language as the email.\nUser Provided a Default Value: null",
                                                        "title": "Observation",
                                                    },
                                                },
                                                "title": "PackingData",
                                                "type": "object",
                                                "description": "Packing details of the good",
                                                "required": [
                                                    "units",
                                                    "packing_type",
                                                    "supplementary_parcels",
                                                    "reasoning___pallets_on_ground",
                                                    "pallets_on_ground",
                                                    "number_eur_pallet",
                                                    "observation",
                                                ],
                                                "additionalProperties": false,
                                            },
                                            "dimensions": {
                                                "properties": {
                                                    "reasoning___loading_meters": {
                                                        "type": "string",
                                                        "description": "A loading meter (LM) is a measurement of the cargo space occupied in a truck, calculated based on the item’s footprint relative to the truck’s standard internal width of 2.4 meters. This measurement helps determine how much linear floor space is needed to transport the goods efficiently.\n\nIf loading meters are provided in the source (often labeled as Mètre linéaire (ML) or Mètre plancher (MPL)), use that value directly, noting it as “Using provided value of X loading meters.”\n\nIf not, calculate as follows:\n\n\t1.\tFor each item: (length × width) / 2.4.\n\t2.\tSum all calculated loading meters.\n\nFor example:\n\n\t•\tItem 1 is 3m × 1.2m, so it takes (3 × 1.2) / 2.4 = 1.5 loading meters.\n\t•\tItem 2 is 2m × 1.6m, so it takes (2 × 1.6) / 2.4 = 1.33 loading meters.\n\nTotal loading meters: 1.5 + 1.33 = 2.83 meters\n\nFor each item, write down the formula, insert values, and compute explicitly.\nTake your time to think rigourously. Do not hesitate to write down intermediate steps. WARNING: VOLUME WILL NOT BE IN ml, this is the unit used for loading meters - ml stands for metres lineaires.",
                                                    },
                                                    "loading_meters": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "Loading meters value\nUser Provided a Default Value: null",
                                                        "title": "Loading Meters",
                                                    },
                                                    "reasoning___volume": {
                                                        "type": "string",
                                                        "description": "Volume represents the total space occupied by an item in cubic meters, calculated as (length × width × height).\n\nIf volume is provided in the source, use that value directly and note it as, “Using provided value of X cubic meters.”\n\nIf not, calculate as follows:\n\n\t1.\tFor each item: length × width × height.\n\t2.\tSum the volumes of all items for the total volume.\n\nExample:\n\n\t•\tItem 1: 2m × 1m × 1.5m = 3 cubic meters\n\t•\tItem 2: 1.5m × 1m × 1m = 1.5 cubic meters\n\nTotal volume: 3 + 1.5 = 4.5 cubic meters\n\nFor each item, write down the formula, insert values, and compute explicitly. Take your time to ensure accuracy, and write intermediate steps if necessary.",
                                                    },
                                                    "volume": {
                                                        "anyOf": [{"type": "number"}, {"type": "null"}],
                                                        "description": "Volume value in cubic meters\nUser Provided a Default Value: null",
                                                        "title": "Volume",
                                                    },
                                                },
                                                "title": "DimensionsData",
                                                "type": "object",
                                                "description": "Dimensions of the good",
                                                "required": ["reasoning___loading_meters", "loading_meters", "reasoning___volume", "volume"],
                                                "additionalProperties": false,
                                            },
                                            "weight": {
                                                "anyOf": [{"type": "number"}, {"type": "null"}],
                                                "description": "Weight of the good (in kilograms)\nUser Provided a Default Value: null",
                                                "title": "Weight",
                                            },
                                            "temperature_infos": {
                                                "anyOf": [
                                                    {
                                                        "properties": {
                                                            "min_temperature": {
                                                                "anyOf": [{"type": "number"}, {"type": "null"}],
                                                                "description": "Minimum temperature (in Celsius)",
                                                                "title": "Min Temperature",
                                                            },
                                                            "max_temperature": {
                                                                "anyOf": [{"type": "number"}, {"type": "null"}],
                                                                "description": "Maximum temperature (in Celsius)",
                                                                "title": "Max Temperature",
                                                            },
                                                            "category": {
                                                                "anyOf": [{"enum": ["Dry", "Fresh", "Frozen"], "type": "string"}, {"type": "null"}],
                                                                "description": "Temperature control requirements",
                                                                "title": "Category",
                                                            },
                                                        },
                                                        "title": "TemperatureInfosData",
                                                        "type": "object",
                                                        "required": ["min_temperature", "max_temperature", "category"],
                                                        "additionalProperties": false,
                                                    },
                                                    {"type": "null"},
                                                ],
                                                "description": "Temperature infos of the good\nUser Provided a Default Value: {\"min_temperature\": null, \"max_temperature\": null, \"category\": null}",
                                            },
                                            "reasoning___dangerous_goods_infos": {
                                                "type": "string",
                                                "description": "Document the list of dangerous goods information found in the source document. Include:\n1. Weight of the dangerous good if present\n2. UN code of the dangerous good (usually a number: UN 1234)\n3. UN label of the dangerous good (usually a name close to the code. E.g., 'UN 1203 ESSENCE' then the label is 'ESSENCE'), if present\n4. UN class of the dangerous good, if present\n5. UN packing group, if present\n6. ADR tunnel code, if present.\n\nYou should make a list like:\n- UN code: 1234, UN label: 'ESSENCE', UN class: 3, UN packing group: II, ADR tunnel code: B\n- UN code: 5678, UN label: 'ACIDE', UN class: 8, UN packing group: I, ADR tunnel code: C\n... and so on.\n\n",
                                            },
                                            "dangerous_goods_infos": {
                                                "description": "Dangerous goods infos of the good\nUser Provided a Default Value: []",
                                                "items": {
                                                    "properties": {
                                                        "weight": {
                                                            "anyOf": [{"type": "number"}, {"type": "null"}],
                                                            "description": "Weight of the dangerous good (in kilograms)\nUser Provided a Default Value: null",
                                                            "title": "Weight",
                                                        },
                                                        "UN_code": {
                                                            "anyOf": [{"type": "number"}, {"type": "null"}],
                                                            "description": "UN code of the dangerous good\nUser Provided a Default Value: null",
                                                            "title": "Un Code",
                                                        },
                                                        "UN_label": {
                                                            "anyOf": [{"type": "string"}, {"type": "null"}],
                                                            "description": "UN label of the dangerous good\nUser Provided a Default Value: null",
                                                            "title": "Un Label",
                                                        },
                                                        "UN_class": {
                                                            "anyOf": [{"type": "string"}, {"type": "null"}],
                                                            "description": "UN class of the dangerous good\nUser Provided a Default Value: null",
                                                            "title": "Un Class",
                                                        },
                                                        "UN_packing_group": {
                                                            "anyOf": [{"enum": ["I", "II", "III"], "type": "string"}, {"type": "null"}],
                                                            "description": "UN packing group of the dangerous good\nUser Provided a Default Value: null",
                                                            "title": "Un Packing Group",
                                                        },
                                                        "ADR_tunnel_code": {
                                                            "anyOf": [
                                                                {"enum": ["B", "B1000C", "B/D", "B/E", "C", "C5000D", "C/D", "C/E", "D", "D/E", "E", "-"], "type": "string"},
                                                                {"type": "null"},
                                                            ],
                                                            "description": "ADR tunnel code of the dangerous good\nUser Provided a Default Value: null",
                                                            "title": "Adr Tunnel Code",
                                                        },
                                                    },
                                                    "title": "DangerousGoodsInfoData",
                                                    "type": "object",
                                                    "required": ["weight", "UN_code", "UN_label", "UN_class", "UN_packing_group", "ADR_tunnel_code"],
                                                    "additionalProperties": false,
                                                },
                                                "title": "Dangerous Goods Infos",
                                                "type": "array",
                                            },
                                        },
                                        "title": "GoodsData",
                                        "type": "object",
                                        "description": "Description of transported goods",
                                        "required": ["packing", "dimensions", "weight", "temperature_infos", "reasoning___dangerous_goods_infos", "dangerous_goods_infos"],
                                        "additionalProperties": false,
                                    },
                                    "reasoning___transport_constraints": {
                                        "type": "string",
                                        "description": "Document the specific facts from the source that determine vehicle requirements. For example:\n- List any explicitly mentioned vehicle specifications\n- Note any temperature requirements stated\n- Record loading/unloading equipment mentioned\n- Include any access restrictions specified\n\nExample: 'Source specifies: refrigerated transport required, -20°C, loading dock not available, tail lift needed.'\n\nOnly include information that is explicitly stated in the source document. Do not make assumptions about requirements that aren't clearly specified.",
                                    },
                                    "transport_constraints": {
                                        "properties": {
                                            "vehicle_type": {
                                                "anyOf": [{"enum": ["Tractor", "Carrier", "Light Vehicle"], "type": "string"}, {"type": "null"}],
                                                "description": "Primary vehicle classification:\n- 'Tractor': for semi-trailer trucks (Tracteur)\n- 'Carrier': for rigid trucks (Porteur) \n- 'Light Vehicle': for vans and smaller commercial vehicles (Véhicule léger)\nUser Provided a Default Value: null",
                                                "title": "Vehicle Type",
                                            },
                                            "body_type": {
                                                "anyOf": [{"enum": ["Tautliner", "Dry Van", "Refrigerated", "Flatbed", "Tanker"], "type": "string"}, {"type": "null"}],
                                                "description": "Type of cargo body/trailer:\n- 'Tautliner': curtainside trailer (Tautliner/Transpalette)\n- 'Dry Van': box trailer (Fourgon)\n- 'Refrigerated': temperature-controlled (Réfrigéré)\n- 'Flatbed': open platform (Plateau)\n- 'Tanker': liquid cargo (Citerne)\nUser Provided a Default Value: null",
                                                "title": "Body Type",
                                            },
                                            "tail_lift": {
                                                "anyOf": [{"type": "boolean"}, {"type": "null"}],
                                                "description": "Indicates if the vehicle is equipped with a hydraulic tail lift for ground-level loading/unloading without dock access (Hayon élévateur)\nUser Provided a Default Value: null",
                                                "title": "Tail Lift",
                                            },
                                            "crane": {
                                                "anyOf": [{"type": "boolean"}, {"type": "null"}],
                                                "description": "Indicates if the vehicle is equipped with a loading crane/HIAB for self-loading and unloading of heavy cargo (Grue)\nUser Provided a Default Value: null",
                                                "title": "Crane",
                                            },
                                        },
                                        "title": "TruckData",
                                        "type": "object",
                                        "description": "List of transport constraints informations",
                                        "required": ["vehicle_type", "body_type", "tail_lift", "crane"],
                                        "additionalProperties": false,
                                    },
                                },
                                "title": "ShipmentData",
                                "type": "object",
                                "required": [
                                    "shipment_id",
                                    "reasoning___sender",
                                    "sender",
                                    "reasoning___recipient",
                                    "recipient",
                                    "reasoning___goods",
                                    "goods",
                                    "reasoning___transport_constraints",
                                    "transport_constraints",
                                ],
                                "additionalProperties": false,
                            },
                            "title": "Shipments",
                            "type": "array",
                        },
                    },
                    "required": ["reasoning___root", "booking_id", "payment", "client", "reasoning___shipments", "shipments"],
                    "title": "RoadBookingConfirmationData",
                    "type": "object",
                    "additionalProperties": false,
                },
                "type": "json_schema",
                "description": null,
                "name": "sch_id_b4b77f8f21c22240",
                "strict": true,
            },
        },
        {
            "type": "literal_error",
            "loc": ["body", "openai_responses_output", "text", "format", "ResponseFormatJSONObject", "type"],
            "msg": "Input should be 'json_object'",
            "input": "json_schema",
            "ctx": {"expected": "'json_object'"},
        },
    ]
}

In [12]:
## This also work with streaming!
from jiter import from_json
from IPython.display import clear_output, display

client = OpenAI()
output_text_cum = ""
with client.responses.stream(
    model="gpt-4o-mini",
    input=schema_obj.openai_responses_input + doc_msg.openai_responses_input,
    text={
        "format": {
            "type": "json_schema",
            "name": schema_obj.id,
            "schema": schema_obj.inference_json_schema,
            "strict": True
        }
    } 
) as stream:
    for event in stream:
        if event.type == "response.output_text.delta":
            output_text_cum += event.delta
            clear_output(wait=True)
            # Use this if you want to parse partial json objects.
            display(from_json(bytes(output_text_cum, "utf-8"), partial_mode="trailing-strings"))
        elif event.type == "response.completed":
            clear_output(wait=True)
            # Should be directly a valid json object!
            display(json.loads(event.response.output_text))


{'reasoning___root': 'Analysis shows:\n- Clear booking confirmation document from ACME CORPORATION\n- Single shipment under booking reference BC-67890\n- Client identified as ACME CORPORATION from letterhead\n- Shipping address and billing address provided\n- All required signatures and VAT information is present',
 'booking_id': 'BC-67890',
 'payment': {'total_price': 1500, 'currency': 'EUR'},
 'client': {'company_name': 'ACME CORPORATION',
  'reasoning___VAT_number': 'Found VAT number GB123456789 in header next to company details',
  'VAT_number': 'GB123456789',
  'city': 'London',
  'postal_code': 'WC2N 5DU',
  'country': 'GB',
  'code': None,
  'email': 'client@acme.com'},
 'reasoning___shipments': 'Shipment Analysis:\n1: Dangerous goods - 5 pallets at a weight of 500 kg (UN1202, liquid, Class 3)\n2: Total volume of 1.44 m³\n3: Temperature range of 5°C to 25°C, Fracht considerations to be observed during the transport',
 'shipments': [{'shipment_id': 'SHIP-001',
   'reasoning___sen